In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from datetime import date, datetime
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
sns.set_theme()
sns.set_color_codes()
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)

# column_names = ['# Seen', 'ALSA', '# Picked', 'ATA', '# GP', 'GP WR', '# OH', 'OH WR', '# GD', 'GD WR', '# GIH', 'GIH WR', '# GND', 'GND WR', 'IWD', 'Color', 'Rarity']

from Utilities import Logger
import WUBRG
from data_graphing.LineColors import LineColors
from game_metadata import SETS, FORMATS
from game_metadata import Card,CardManager, SetMetadata, FormatMetadata
from data_fetching import DataLoader, LoadedData, DataFramer, FramedData, SetManager, CentralManager
from data_fetching.utils.date_helper import get_next_17lands_update_time, get_prev_17lands_update_time

LOAD_ALL = False
TRGT_SET = 'SNC'
LOG_LEVEL = Logger.FLG.DEFAULT
LOG_LEVEL

<Flags.DEFAULT: 3>

In [3]:
print(f"Current Local Time:  {datetime.now()}")
print(f"Last 17Lands Update: {get_prev_17lands_update_time()}")
print(f"Current UTC Time:    {datetime.utcnow()}")
print(f"Next 17Lands Update: {get_next_17lands_update_time()}")

Current Local Time:  2022-05-03 11:25:28.769952
Last 17Lands Update: 2022-05-03 02:00:00
Current UTC Time:    2022-05-03 13:55:28.769952
Next 17Lands Update: 2022-05-04 02:00:00


# Objects

In [4]:
import sys
try:
    del sys.modules["data_graphing"] 
    del data_graphing
except:
    pass

from data_graphing import ColorHandler, PlotterHelper

In [5]:
class FramedDataFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        self._ARCH_FUNCS = ArchFuncs(self._DATA)
        self._SNGL_CARD_FUNCS = SingleCardFuncs(self._DATA)
    
    @property
    def SET(self):
        """The draft set."""
        return self._DATA.SET
    
    @property
    def FULL_SET(self):
        """The full name of the draft set."""
        return self._DATA.FULL_SET
    
    @property
    def FORMAT(self):
        """The format type."""
        return self._DATA.FORMAT
    
    @property
    def SHORT_FORMAT(self):
        """The shorthand of the format type."""
        return self._DATA.FULL_FORMAT
    
    @property
    def DATA(self):
        """The object which contains the data about the set and format."""
        return self._DATA
        
    def check_for_updates(self):
        """Populates and updates all data properties, filling in missing data."""
        self._DATA.check_for_updates()

    def reload_data(self):
        """Populates and updates all data properties, reloading all data."""
        self._DATA.reload_data()  
    
    def deck_group_frame(self, name=None, date=None, summary=False):
        """Returns a subset of the 'GROUPED_ARCHETYPE' data as a DataFrame."""
        return self.DATA.deck_group_frame(name, date, summary)
     
    def deck_archetype_frame(self, deck_color=None, date=None, summary=False):
        """Returns a subset of the 'SINGLE_ARCHETYPE' data as a DataFrame."""
        return self.DATA.deck_archetype_frame(deck_color, date, summary)
    
    def card_frame(self, name=None, deck_color=None, date=None, card_color=None, card_rarity=None, summary=False):
        """Returns a subset of the 'CARD' data as a DataFrame."""
        return self.DATA.card_frame(name, deck_color, date, card_color, card_rarity, summary)
    
    def compress_date_range_data(self, start_date, end_date, card_name=None):
        """Summarizes card data over a provided set of time."""
        return self.DATA.compress_date_range_data(start_date, end_date, card_name)
        
    
    #ArchFuncs
    def get_games_played(self, deck_color):
        return self._ARCH_FUNCS.get_games_played(deck_color)
        
    def get_avg_winrate(self, day=None, arch='All Decks'):
        return self._ARCH_FUNCS.get_avg_winrate(day, arch)
    
    def get_archetype_frame(self, colors, roll=None):
        return self._ARCH_FUNCS.get_archetype_frame(colors, roll)

    def get_archetype_winrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.get_archetype_winrate_history(color_filter, roll)

    def plot_archetype_winrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.plot_archetype_winrate_history(color_filter, roll)       
        
    def get_archetype_playrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.get_archetype_playrate_history(color_filter, roll)

    def plot_archetype_playrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.plot_archetype_playrate_history(color_filter, roll)
    
    
    #SingleCardFuncs
    def get_card_summary(self, card_name, colors='', roll=None):
        return self._SNGL_CARD_FUNCS.get_card_summary(card_name, colors, roll)

    def plot_card_summary(self, card_name, colors='', roll=None):
        return self._SNGL_CARD_FUNCS.plot_card_summary(card_name, colors, roll)
    
    def get_pick_stats(self, card_name, roll=None):
        return self._SNGL_CARD_FUNCS.get_pick_stats(card_name, roll)
    
    def plot_pick_stats(self, card_name, roll=None):
        return self._SNGL_CARD_FUNCS.plot_pick_stats(card_name, roll)
    
    def card_archetype_performance(self, card_name):
        return self._SNGL_CARD_FUNCS.card_archetype_performance(card_name)
    
    def stat_archetype_performance(self, stat_name, color_cols=None, min_colors=0, max_colors=5):
        return self._SNGL_CARD_FUNCS.stat_archetype_performance(stat_name, color_cols, min_colors, max_colors)

##    # filters = ['exact', 'subset', 'contains', 'adjascent'*]
##    ## TODO: Determine how to handle colourless cards.
##    ## 'exact': 'U' --> 'U'
##    ## 'subset': 'UW' --> 'U', 'W', 'WU'
##    ## 'contains': 'U' --> 'U', 'UW', 'UB', 'UR', 'UG'...
##    ## 'contains': 'UW' --> 'UW', 'UBW', 'URW', 'UGW'...
##    ## 'superset': 'UW' --> 'UW', 'UBW', 'URW', 'UGW'...
##    ## 'adjascent': 'UW' --> 'U', 'W', 'UW', 'UG', 'WG', 'UWG'...
##    def card_color_filter(frame, card_color=None, filter_style=''):
##        if card_color is None: 
##            return frame
##        
##            card_color = WUBRG.get_color_identity(card_color)
##            frame = frame[frame['Color'] == card_color]
##            return frame

    
    def compare_card_evaluations(self, start_date, end_date):
        def inner_func(date):
            df = self.card_frame(date=date, deck_color='')
            df.index = [tup[2] for tup in df.index]
            return df

        first = inner_func(date=start_date)
        last = inner_func(date=end_date)
        diff = last[['ALSA', 'ATA', 'Color', 'Rarity']].copy()
        diff['Δ ALSA'] = first['ALSA'] - last['ALSA']
        diff['Δ ATA'] = first['ATA'] - last['ATA']
        return diff[['ALSA', 'Δ ALSA', 'ATA', 'Δ ATA', 'Color', 'Rarity']]


    def get_top(self, column, count=10, asc=True, card_color=None, card_rarity=None, deck_color='', play_lim=None):
        frame = self.card_frame(deck_color=deck_color, summary=True, card_rarity=card_rarity)
        frame = frame.sort_values(column, ascending=asc)

        if card_color is not None:
            card_color = WUBRG.get_color_identity(card_color)
            frame = frame[frame['Color'] == card_color]

        if play_lim is not None:
            if type(play_lim) is float: play_lim *= self.get_games_played(deck_color)
            print(f'Minimum Games played to be included: {play_lim}')
            frame = frame[frame['# GP'] >= play_lim]

        return frame.head(count)

In [6]:
from data_graphing import ROLL
from WUBRG.consts import COLOR_PAIRS

class ArchFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        
    def get_games_played(self, deck_color):
        if deck_color: return self._DATA.deck_archetype_frame(deck_color=deck_color, summary=True)['Games'].sum()
        else: return self._DATA.deck_group_frame(name='All Decks', summary=True)['Games']
    
    def get_avg_winrate(self, day=None, arch='All Decks'):
        if day: return self._DATA.deck_group_frame(date=day, summary=False).loc[(day, arch)]['Win %']
        else: return self._DATA.deck_group_frame(date=day, summary=True).loc[arch]['Win %']   
    
    def get_archetype_frame(self, colors, roll=None):
        if roll is None: roll = ROLL
        win_rate_frame = self._DATA.deck_archetype_frame(deck_color=colors)
        win_rate_frame.index = [tup[0] for tup in win_rate_frame.index]
        #win_rate_frame = win_rate_frame[['Splash', 'Games', 'Win %']]
        win_rate_frame = win_rate_frame[win_rate_frame['Splash'] == False][['Wins', 'Games']]
        rolling = win_rate_frame.rolling(window=roll, min_periods=1, center=True).mean().round()
        rolling['Win %'] = round((rolling['Wins'] / rolling['Games']) * 100, 2)
        rolling['Avg. Win%'] = [self.get_avg_winrate(idx) for idx in win_rate_frame.index]
        rolling['2C Win%'] = [self.get_avg_winrate(idx, arch='Two-color') for idx in win_rate_frame.index]
        rolling['Win % Offset'] = rolling['Win %'] - rolling['Avg. Win%']
        return rolling

    def get_archetype_winrate_history(self, color_filter=None, roll=None):
        if roll is None: roll = ROLL
        d = dict()
        
        if isinstance(color_filter, list):
            color_list = color_filter
        else:
            color_list = COLOR_PAIRS
        
        for col in color_list:
            temp_frame = self.get_archetype_frame(col)
            d[col] = temp_frame['Win %']
        d['AVG'] = temp_frame ['Avg. Win%']
        d['2C'] = temp_frame ['2C Win%']

        test_frame = pd.DataFrame.from_dict(d)
        test_frame.index = [idx[5:] for idx in test_frame.index]
        if isinstance(color_filter, str):
            col_filt = [col for col in COLOR_PAIRS if color_filter in col] + ['AVG', '2C'] 
            test_frame = test_frame[col_filt]

        rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
        return rolling

    def plot_archetype_winrate_history(self, color_filter=None, roll=None):
        if roll is None: roll = ROLL
            
        test_frame = self.get_archetype_winrate_history(color_filter, roll)
        #lc = LineColors()
        title = f"Archetype Winrates (from 17Lands)\n{self._DATA.SET} - {self._DATA.FORMAT}"

        col_filt = f"Color Filter: {color_filter}"
        rol_filt = f"Rolling Average: {roll} Days"

        if color_filter and roll > 1:
            title += f"\n{col_filt}  -  {rol_filt}"
        elif color_filter:
            title += f"\n{col_filt}"
        elif roll > 1:
            title += f"\n{rol_filt}"
        
        #test_frame.plot(figsize=(20, 10), color=lc.get_col_array(color_filter), title=title, lw=2.5, grid=True)
        test_frame.plot(figsize=(20, 10), title=title, lw=2.5, grid=True)
        plt.xlabel("Date")
        plt.ylabel("Win Percent")
        plt.savefig(f"win_rates_{roll}day_avg.png")
        
        
    #TODO: Implement a more generic version of this that takes in a list of deck colours to include as output. 
    def get_archetype_playrate_history(self, color_filter=None, roll=None):
        if roll is None: roll = ROLL
        d = dict()
        
        if isinstance(color_filter, list):
            color_list = color_filter
        else:
            color_list = COLOR_PAIRS
        
        for col in color_list:
            d[col] = self.get_archetype_frame(col)['Games']

        test_frame = pd.DataFrame.from_dict(d)
        test_frame.index = [idx[5:] for idx in test_frame.index]
        rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
        total = rolling.sum(axis=1)
        playrate = rolling.divide(list(total),axis=0) * 100

        if isinstance(color_filter, str):
            col_filt = [col for col in COLOR_PAIRS if color_filter in col]
            playrate = playrate[col_filt]

        return playrate

    def plot_archetype_playrate_history(self, color_filter=None, roll=None):
        if roll is None: roll = ROLL
        test_frame = self.get_archetype_playrate_history(color_filter, roll)
        #lc = LineColors()
        title = f"Archetpye Playrates (from 17Lands)\n{self._DATA.SET} - {self._DATA.FORMAT}"

        col_filt = f"Color Filter: {color_filter}"
        rol_filt = f"Rolling Average: {roll} Days"

        if color_filter and roll > 1:
            title += f"\n{col_filt}  -  {rol_filt}"
        elif color_filter:
                title += f"\n{col_filt}"
        elif roll > 1:
                title += f"\n{rol_filt}"
        #test_frame.plot(figsize=(20, 10), color=lc.get_col_array(color_filter), title=title, lw=2.5, grid=True)
        test_frame.plot(figsize=(20, 10), title=title, lw=2.5, grid=True)
        plt.xlabel("Date")
        plt.ylabel("Percent of Metagame")
        plt.savefig(f"play_rates_{roll}day_avg.png")

In [7]:
from data_graphing import ROLL
from WUBRG import COLOR_COMBINATIONS
from WUBRG.consts import COLOR_PAIRS


class SingleCardFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        self._COLOR_IDX = 0
        
    def _shorten_data(self, card_name, roll, cols, colors=''):
        frame = self._DATA.card_frame(name=card_name, deck_color=colors)[cols]
        frame.index = [tup[0][5:] for tup in frame.index]
        rolling = frame.rolling(window=roll, min_periods=1, center=True).mean()
        return rolling

    def plot_card_summary(self, card_name, colors='', roll=None):
        if roll is None: roll = ROLL        
        rolling = self._shorten_data(card_name, roll, ['GIH WR', 'GND WR', 'ATA', 'ALSA', '# GP', '# GIH', '# Picked', '# Seen'], colors=colors)
        
        color_dict = {
            'GIH WR' : (0.33, 0.66, 0.41, 0.9),
            'GND WR' : (0.77, 0.31, 0.32, 0.9),
            'ATA' : (0.87, 0.52, 0.32, 0.9),
            'ALSA' : (0.3, 0.45, 0.69, 0.9),
            '# GP' : (0.51, 0.45, 0.7, 0.9),
            '# GIH' : (0.33, 0.66, 0.41, 0.9),
            '# Picked' : (0.87, 0.52, 0.32, 0.9),
            '# Seen' : (0.3, 0.45, 0.69, 0.9)
        }
        
        plot_help = PlotterHelper(self._DATA, color_dict=color_dict)
        fig, ax = plot_help.new_quad_plot(card_name)
        plot_help.accredit()
        plot_help.desc_note(colors=colors, roll=roll)
                
        plot_help.set_labels(y_label="Win Percent", g_x=0, g_y=0)
        plot_help.set_data(rolling, ['GIH WR', 'GND WR'], g_x=0, g_y=0)
        
        plot_help.set_labels(y_label="Pick Number", g_x=0, g_y=1)
        plot_help.set_data(rolling, ['ALSA', 'ATA'], inv_y=True, g_x=0, g_y=1)
        
        plot_help.set_labels(x_label="Date", y_label="# of Games", g_x=1, g_y=0)
        plot_help.set_data(rolling, ['# GP', '# GIH'], g_x=1, g_y=0)

        plot_help.set_labels(x_label="Date", y_label="# of Cards", g_x=1, g_y=1)
        plot_help.set_data(rolling, ['# Seen', '# Picked'], g_x=1, g_y=1)
        
        
        plot_help.save_fig(f"pcs_{card_name}_{colors}.png")
    
    
    def plot_pick_stats(self, card_name, roll=None):
        if roll is None: roll = ROLL        
        taken_data = self._shorten_data(card_name, roll, ['ALSA', 'ATA'])
        
        plot_help = PlotterHelper(self._DATA)
        fig, ax = plot_help.new_single_plot(card_name)
        plot_help.accredit()
        plot_help.desc_note(roll=roll)
        
        plot_help.set_labels(x_label="Date", y_label="Pick Number")
        plot_help.set_data(taken_data, ['ALSA', 'ATA'], inv_y=True)
        
        plot_help.save_fig(f"pps_{card_name}.png")


    def card_archetype_performance(self, card_name):
        d = dict()
        d['AVG'] = self._DATA.card_frame(card_name, '', summary=True)
        for col in COLOR_PAIRS:
            d[col] = self._DATA.card_frame(card_name, col, summary=True)

        test_frame = pd.DataFrame.from_dict(d).T
        return test_frame
    
    
    def stat_archetype_performance(self, stat_name, color_cols=None, min_colors=0, max_colors=5):
        series = self._DATA.card_frame(summary=True)[stat_name]
        frame = series.reset_index(level=0)
        ret = pd.pivot_table(frame, index='Name', columns='Deck Colors')
        ret.columns = ret.columns.droplevel(0)
        #ret = ret[COLOR_COMBINATIONS]  #Re-orders the columns in WUBRG order
        if color_cols is not None:
            ret = ret[color_cols]
        return ret

# Initialization

In [8]:
if LOAD_ALL:
    start = datetime.utcnow()
    data_manager = CentralManager()
    data_manager.check_for_updates()
    end = datetime.utcnow()
    print(f"\n --- Data loaded in {end - start}.")

    SNC = data_manager['SNC']
    NEO = data_manager['NEO']
    VOW = data_manager['VOW']
    MID = data_manager['MID']
    set_data = data_manager[TRGT_SET]
else:
    set_data = SetManager(TRGT_SET)
    set_data.check_for_updates()
    
examiner = FramedDataFuncs(set_data.BO1)

Loading set metadata for: SNC
Fetching data for set: SNC
Fetching card data for set: SNC
Done!

Checking for missing data for SNC PremierDraft...
Getting overall data for SNC PremierDraft
Getting data for SNC PremierDraft, date: 2022-04-28
Getting data for SNC PremierDraft, date: 2022-04-29
Getting data for SNC PremierDraft, date: 2022-04-30
Getting data for SNC PremierDraft, date: 2022-05-01
Getting data for SNC PremierDraft, date: 2022-05-02
Finished checking for missing data for SNC PremierDraft.

Checking for missing data for SNC TradDraft...
Getting overall data for SNC TradDraft
Getting data for SNC TradDraft, date: 2022-04-28
Getting data for SNC TradDraft, date: 2022-04-29
Getting data for SNC TradDraft, date: 2022-04-30
Getting data for SNC TradDraft, date: 2022-05-01
Getting data for SNC TradDraft, date: 2022-05-02
Finished checking for missing data for SNC TradDraft.

Checking for missing data for SNC QuickDraft...
SNC QuickDraft has no summary data to get!
SNC QuickDraft ha

# Examine Data

In [ ]:
set_data.BO1.DATA.CARD_SUMMARY_FRAME

In [ ]:
examiner.plot_card_summary('Girder Goons')

In [ ]:
examiner._SNGL_CARD_FUNCS._shorten_data('Girder Goons', 3, ['GIH WR', 'GND WR', 'ATA', 'ALSA', '# GP', '# GIH', '# Picked', '# Seen'])

In [ ]:
examiner.plot_pick_stats('Girder Goons')

In [ ]:
examiner.card_archetype_performance('Girder Goons')

In [ ]:
examiner.stat_archetype_performance('GIH WR', color_cols=['WU', 'WG', 'UB', 'BR', 'RG'])

In [ ]:
raise Exception('Stopping Auto-Run!')

## Current Tests

In [ ]:
def gt(x, y):  #Used for function pointer shenanigans
    return x > y

def lt(x, y):  #Used for function pointer shenanigans
    return x < y

def filter_quadrant_cards_df(func1, func2, iwd_thresh=0, play_lim=0.01, card_rarity=None, deck_color=''):
    frame = set_data.BO1.card_frame(deck_color=deck_color, summary=True, card_rarity=card_rarity)
    
    # TODO: Make the mean different based on card rarity so cards aren't moved into incorrect categories.
    
    if play_lim is not None:
        if type(play_lim) is float: play_lim *= set_data.BO1.get_games_played(deck_color)
        print(f'Minimum Games played to be included: {play_lim}')
        frame = frame[frame['# GP'] >= play_lim]
        
    games_played_mean = frame['# GP'].mean()
    cards = frame[func1(frame['IWD'], iwd_thresh)]
    cards = cards[func2(cards['# GP'], games_played_mean)]
    cards = cards.sort_values('IWD', ascending=func1==lt)
    return cards


def get_trap_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(lt, gt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_niche_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(gt, lt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_staple_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(gt, gt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_dreg_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(lt, lt, iwd_thresh, play_lim, card_rarity, deck_color)

In [ ]:
get_niche_cards(deck_color='GW', card_rarity='CU')

In [ ]:
set_data.BO1.card_archetype_performance("Era of Enlightenment")

In [9]:
import functools

@functools.singledispatch
def get_date_slice(arg):
    # By default, if we don't have a special way to handle a parameter, don't change it.
    return arg


@get_date_slice.register(type(None))
def _get_date_slice_none(val):
    # If the parameter is a None, marshall it into a slice.
    return slice(val)


@get_date_slice.register(str)
def _get_date_slice_string(val):
    # If the parameter is a string, marshall it into a single element list to filter properly.
    return [get_color_identity(val)]


@get_date_slice.register(list)
@get_date_slice.register(dict)
@get_date_slice.register(set)
def _get_date_slice_iterable(val):
    # If the parameter is iterable, first convert it to a set to remove duplicates, then handle.
    return [str_date for str_date in val]


@get_date_slice.register(tuple)
def _get_date_slice_range(val):
    # If the parameter is iterable, check if it should be a range.
    if len(val) == 2:
        # If it should be a range, get the colours check that the colours elements are properly ordered.
        str_date1 = val[0]
        str_date2 = val[1]
        return slice(col1, col2)
        
        # TODO: Check date ordering
        # Then make sure a valid range is returned.
        if color_compare(col2, col1) == 1:
            return slice(col1, col2)
        else:
            # TODO: See if returning a reversed range is possible.
            return slice(col2, col1)
    else:
        # If not, handle it like as an iterable.
        return _get_date_slice_iterable(val)

In [13]:
set_data.BO1.DATA.SINGLE_ARCHETYPE_SUMMARY_FRAME.loc[['W']]

,Colors,Splash,Wins,Games,Win %
Name,,,,,
W,W,False,22,38,57.890
W,W,True,16,27,59.260


In [14]:
from data_fetching.utils.index_slice_helper import get_name_slice, get_color_slice

In [16]:
set_data.BO1.DATA.CARD_HISTORY_FRAME.loc[slice('2022-04-29', '2022-05-01'), get_color_slice(''), get_name_slice({'Girder Goons', 'Angelic Observer', 'A Little Chat', 'Case the Joint'})]

# Seen  ALSA  # Picked    ATA   # GP  \
Date       Deck Colors Name                                                     
2022-04-29             Angelic Observer    4217 4.901       577  6.735   1931   
                       Case the Joint     15950 8.124      1696 11.900   1799   
                       A Little Chat       4676 5.268       817  7.268   2875   
                       Girder Goons       12674 5.131      3282  6.408  16068   
2022-04-30             Angelic Observer    3858 4.841       540  6.967   1856   
                       Case the Joint     13973 8.172      1575 11.832   1711   
                       A Little Chat       4204 5.290       755  7.181   2941   
                       Girder Goons       11059 4.876      2797  6.075  14975   
2022-05-01             Angelic Observer    4181 4.965       605  7.126   2100   
                       Case the Joint     15569 8.208      1747 11.887   1628   
                       A Little Chat       4559 5.238       811  7.083   3289   
                       Girder Goons       11548 4.838      2973  6.180  16033   

                                         GP WR  # OH  OH WR  # GD  GD WR  \
Date       Deck Colors Name                                                
2022-04-29             Angelic Observer 53.496   313 51.438   537 52.700   
                       Case the Joint   54.697   304 50.329   536 57.836   
                       A Little Chat    56.452   468 55.769   868 57.488   
                       Girder Goons     57.070  2670 57.116  4458 60.655   
2022-04-30             Angelic Observer 54.095   309 53.398   523 51.052   
                       Case the Joint   53.536   277 49.097   526 54.753   
                       A Little Chat    57.973   506 51.581   854 60.070   
                       Girder Goons     56.868  2456 56.596  4317 60.343   
2022-05-01             Angelic Observer 51.667   315 45.079   557 55.655   
                       Case the Joint   51.781   261 50.958   487 50.103   
                       A Little Chat    57.099   585 57.094   983 60.834   
                       Girder Goons     55.760  2705 57.560  4620 58.420   

                                         # GIH GIH WR  # GND GND WR    IWD  \
Date       Deck Colors Name                                                  
2022-04-29             Angelic Observer    850 52.235   1088 54.228 -1.993   
                       Case the Joint      840 55.119    961 54.318  0.801   
                       A Little Chat      1336 56.886   1540 56.104  0.782   
                       Girder Goons       7128 59.329   8967 55.280  4.049   
2022-04-30             Angelic Observer    832 51.923   1030 55.922 -3.999   
                       Case the Joint      803 52.802    909 54.235 -1.433   
                       A Little Chat      1360 56.912   1583 58.939 -2.027   
                       Girder Goons       6773 58.984   8217 55.093  3.891   
2022-05-01             Angelic Observer    872 51.835   1236 51.618  0.217   
                       Case the Joint      748 50.401    880 52.955 -2.553   
                       A Little Chat      1568 59.439   1721 54.968  4.471   
                       Girder Goons       7325 58.102   8741 53.747  4.356   

                                        Rarity Color Cast Color   CMC  \
Date       Deck Colors Name                                             
2022-04-29             Angelic Observer      U     W          W 6.000   
                       Case the Joint        C     U          U 4.000   
                       A Little Chat         U     U          U 2.000   
                       Girder Goons          C     B          B 5.000   
2022-04-30             Angelic Observer      U     W          W 6.000   
                       Case the Joint        C     U          U 4.000   
                       A Little Chat         U     U          U 2.000   
                       Girder Goons          C     B          B 5.000   
2022-05-01        

In [ ]:
data = set_data.BO1.DATA.CARD_HISTORY_FRAME.loc(axis=0)[pd.IndexSlice[slice(None), get_color_slice(''), get_name_slice(None)]]
cheap = data[data['CMC'] <= 3]
expensive = data[data['CMC'] >= 5]

In [ ]:
cheap['Games Won'] = cheap['# GIH'] * cheap['GIH WR'] * 0.01
cheap_sums = cheap[['# GP', '# GIH', 'Games Won']].sum()
cheap_wr = cheap_sums['Games Won'] / cheap_sums['# GIH']
cheap_wr

In [ ]:
expensive['Games Won'] = expensive['# GIH'] * expensive['GIH WR'] * 0.01
expensive_sums = expensive[['# GP', '# GIH', 'Games Won']].sum()
expensive_wr = expensive_sums['Games Won'] / expensive_sums['# GIH']
expensive_wr

In [ ]:
for i in range(0, 9):
    by_cmc = data[data['CMC'] == i]
    by_cmc['Games Won'] = by_cmc['# GIH'] * by_cmc['GIH WR'] * 0.01
    sums = by_cmc[['# GP', '# GIH', 'Games Won']].sum()
    avg = by_cmc[['GIH WR']].mean()['GIH WR']
    wr = sums['Games Won'] / sums['# GIH'] * 100
    print(f"CMC: {i}, WR: {wr}  AVG: {avg}")

# TODO

- Calculate archetype openess
 - GIH WR & ALSA based
 - 2.25 of a common per draft 
- Improve graphing capabilities
 - Modify Graphs so they also save an image to a cache folder.
 - Implement better graph titles and axes
 - Update functions to be more general, and have DataFrames piped into them.
- Move sets of files into subfolders based on functionality and relationship (DataFetch, Cards, Graphing, Utils, etc.)
- Better group settings and consts into one file/location.

# Data Graphing and Display

## Card Summary

In [ ]:
examiner.plot_card_summary('Jewel Thief')

In [ ]:
examiner.plot_card_summary('Disciplined Duelist')

In [ ]:
examiner.get_top('GIH WR', count=25, asc=False, deck_color='', card_color=None, card_rarity='C', play_lim=0.005)

## Card Pick Order Stats

In [ ]:
examiner.plot_pick_stats('Inspiring Overseer')

In [ ]:
examiner.plot_pick_stats('Inspiring Overseer', 3)

## Archetype Winrate History

In [ ]:
examiner.get_archetype_winrate_history()

In [ ]:
examiner.plot_archetype_winrate_history(['WU', 'WG', 'UB', 'RB', 'RG'])

## Archetype Playrate History

In [ ]:
examiner.get_archetype_playrate_history()

In [ ]:
examiner.plot_archetype_playrate_history(['WU', 'WG', 'UB', 'RB', 'RG'])

## Card Pick Order Changes

In [ ]:
diff = examiner.compare_card_evaluations('2022-04-28', '2022-05-02')
commons = diff[diff['Rarity'] == 'C']
uncommons = diff[diff['Rarity'] == 'U']

In [ ]:
SRT_TRG = 'Δ ATA'
commons.sort_values(SRT_TRG, ascending=False).head(20)

In [ ]:
commons.sort_values(SRT_TRG, ascending=True).head(20)

In [ ]:
uncommons.sort_values(SRT_TRG, ascending=False).head(10)

In [ ]:
uncommons.sort_values(SRT_TRG, ascending=True).head(10)